In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [21]:
class objs():
    def __init__(self, x = 0, y = 0, size = 50):
        self.x, self.y = x, y
        self.size = size

class predator(objs):
    def __init__(self, feeding = 100):
        super().__init__()
        self.feeding = feeding

In [22]:
a = objs()

In [26]:
a.x, a.y, a.size

(0, 0, 50)

In [24]:
b = predator()

In [25]:
b.x, b.y, b.size, b.feeding

(0, 0, 50, 100)